In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join('..', 'src')))
from funkwpap import *
import sympy as sp, pandas as pd, numpy as np, tqdm, time, sys, matplotlib.pyplot as plt, tqdm
from statistics import mean
from scipy.optimize import fsolve
import gurobipy as gb


In [2]:
import sympy as sp

# Define symbolic variable
x = sp.symbols('x')

# Create a Regulator with an emission cap
RegulatorETS = Regulator("ETS", permit_price=20.0, emission_cap=1000000)

# Define Countries with updated names and attributes
country1 = Country('Atlantis', size=50, regulator=RegulatorETS, GDP_per_capita=400, population=100000000, industry_percentage=30)
country2 = Country('Omashu', size=20, regulator=RegulatorETS, GDP_per_capita=350, population=50000000, industry_percentage=25)
country3 = Country('Hogsmeade', size=10, regulator=RegulatorETS, GDP_per_capita=450, population=20000000, industry_percentage=20)
country4 = Country('The_Court_of_Miracles', size=5, regulator=RegulatorETS, GDP_per_capita=300, population=10000000, industry_percentage=15)
country5 = Country('Lilipoupoli', size=2, regulator=RegulatorETS, GDP_per_capita=250, population=5000000, industry_percentage=10)

# Define Sectors with demand functions
sector1 = Sector('Steel', price_demand_function=200 - 0.1*x, free_emission_multiplier=0.8, regulator=RegulatorETS)
sector2 = Sector('Cement', price_demand_function=150 - 0.05*x, free_emission_multiplier=0.85, regulator=RegulatorETS)
sector3 = Sector('Paper', price_demand_function=100 - 0.02*x, free_emission_multiplier=0.05, regulator=RegulatorETS)
sector4 = Sector('Chemicals', price_demand_function=250 - 0.15*x, free_emission_multiplier=0.9, regulator=RegulatorETS)
sector5 = Sector('Automotive', price_demand_function=300 - 0.2*x, free_emission_multiplier=0.75, regulator=RegulatorETS)
sector6 = Sector('Textiles', price_demand_function=80 - 0.01*x, free_emission_multiplier=0, regulator=RegulatorETS)

# Create Firms with updated definitions (production_cost_function=0, abatement_cost_function as previous production_cost_function)
# Firms in Sector 1: Steel
firm_S1_F1 = Firm('S1_F1', sector1, country1, x*0, 2*x + 3*x**2 + x**3, 10000, 5000, 0, RegulatorETS)
firm_S1_F2 = Firm('S1_F2', sector1, country1, x*0, 3*x + 2*x**2 + 2*x**3, 12000, 6000, 0, RegulatorETS)
firm_S1_F3 = Firm('S1_F3', sector1, country2, x*0, 4*x + x**2 + 3*x**3, 8000, 4000, 0, RegulatorETS)
firm_S1_F4 = Firm('S1_F4', sector1, country3, x*0, 2*x + 2*x**2 + 2*x**3, 7000, 3500, 0, RegulatorETS)
firm_S1_F5 = Firm('S1_F5', sector1, country4, x*0, 3*x + 3*x**2 + x**3, 6000, 3000, 0, RegulatorETS)

# Firms in Sector 2: Cement
firm_S2_F1 = Firm('S2_F1', sector2, country1, x*0, x + 2*x**2 + 3*x**3, 9000, 4500, 0, RegulatorETS)
firm_S2_F2 = Firm('S2_F2', sector2, country1, x*0, 2*x + x**2 + 2*x**3, 9500, 4750, 0, RegulatorETS)
firm_S2_F3 = Firm('S2_F3', sector2, country2, x*0, 3*x + x**2 + x**3, 7000, 3500, 0, RegulatorETS)
firm_S2_F4 = Firm('S2_F4', sector2, country3, x*0, x + 3*x**2 + x**3, 6000, 3000, 0, RegulatorETS)
firm_S2_F5 = Firm('S2_F5', sector2, country4, x*0, 2*x + 2*x**2 + x**3, 5000, 2500, 0, RegulatorETS)

#Firms in Sector 3: Paper
firm_S3_F1 = Firm('S3_F1', sector3, country3, x*0, x + x**2 + x**3, 5000, 10000, 0, RegulatorETS)
firm_S3_F2 = Firm('S3_F2', sector3, country4, x*0, x + 2*x**2 + x**3, 4500, 9000, 0, RegulatorETS)
firm_S3_F3 = Firm('S3_F3', sector3, country5, x*0, x + x**2 + 2*x**3, 3000, 6000, 0, RegulatorETS)
firm_S3_F4 = Firm('S3_F4', sector3, country2, x*0, 2*x + x**2 + x**3, 5500, 11000, 0, RegulatorETS)
firm_S3_F5 = Firm('S3_F5', sector3, country1, x*0, x + 2*x**2 + 2*x**3, 6000, 12000, 0, RegulatorETS)

# Firms in Sector 4: Chemicals
firm_S4_F1 = Firm('S4_F1', sector4, country1, x*0, 3*x + 4*x**2 + x**3, 11000, 5500, 0, RegulatorETS)
firm_S4_F2 = Firm('S4_F2', sector4, country1, x*0, 4*x + 3*x**2 + 2*x**3, 10500, 5250, 0, RegulatorETS)
firm_S4_F3 = Firm('S4_F3', sector4, country2, x*0, 2*x + 5*x**2 + x**3, 9000, 4500, 0, RegulatorETS)
firm_S4_F4 = Firm('S4_F4', sector4, country3, x*0, 3*x + 3*x**2 + 3*x**3, 8000, 4000, 0, RegulatorETS)
firm_S4_F5 = Firm('S4_F5', sector4, country4, x*0, 2*x + 2*x**2 + 4*x**3, 7000, 3500, 0, RegulatorETS)

# Firms in Sector 5: Automotive
firm_S5_F1 = Firm('S5_F1', sector5, country1, x*0, 5*x + 4*x**2 + x**3, 12000, 6000, 0, RegulatorETS)
firm_S5_F2 = Firm('S5_F2', sector5, country2, x*0, 4*x + 5*x**2 + 2*x**3, 10000, 5000, 0, RegulatorETS)
firm_S5_F3 = Firm('S5_F3', sector5, country3, x*0, 3*x + 3*x**2 + 2*x**3, 9000, 4500, 0, RegulatorETS)
firm_S5_F4 = Firm('S5_F4', sector5, country4, x*0, 2*x + 4*x**2 + 3*x**3, 8000, 4000, 0, RegulatorETS)
firm_S5_F5 = Firm('S5_F5', sector5, country5, x*0, x + 3*x**2 + 4*x**3, 7000, 3500, 0, RegulatorETS)

# Firms in Sector 6: Textiles
firm_S6_F1 = Firm('S6_F1', sector6, country5, x*0, x + x**2 + x**3, 2000, 10000, 0, RegulatorETS)
firm_S6_F2 = Firm('S6_F2', sector6, country4, x*0, x + 2*x**2 + x**3, 2500, 12500, 0, RegulatorETS)
firm_S6_F3 = Firm('S6_F3', sector6, country3, x*0, x + x**2 + 2*x**3, 3000, 15000, 0, RegulatorETS)
firm_S6_F4 = Firm('S6_F4', sector6, country2, x*0, 2*x + x**2 + x**3, 3500, 17500, 0, RegulatorETS)
firm_S6_F5 = Firm('S6_F5', sector6, country1, x*0, x + 2*x**2 + 2*x**3, 4000, 20000, 0, RegulatorETS)


In [3]:
RegulatorETS.BAU_calculator(precision = 0.01, print_diff = True)
#print(RegulatorETS.BAU_emissions)
# 17638.87773  3750 for 2
RegulatorETS.emission_cap = RegulatorETS.BAU_emissions*0.8
# Optimal solution found for 2 sectors.
# Firm S1_F1 has output 451.62827788960453 and emission 446.7349764049005
# Firm S1_F2 has output 451.6282778896044 and emission 447.8616178523027
# Firm S1_F3 has output 451.6282778896045 and emission 448.41767394370135
# Firm S2_F1 has output 556.5131115584181 and emission 553.3583289997191
# Firm S2_F2 has output 556.513111558418 and emission 552.5696348002654
# Firm S2_F3 has output 556.5131115584182 and emission 551.0577679991109
# Permit price: 103.19300716884365

Set parameter Username
Academic license - for non-commercial use only - expires 2025-08-14


Max diff: 0.01, permit price = 20.0, cap = 1000000, second_stage = False, a = 1.000000

In [4]:
RegulatorETS.optimization_with_positive_constraints_ab(print_output= True, gurobi_print=False)
RegulatorETS.equilibrium_tester(full_output=True)

Optimal solution found
Firm S1_F1 has output 324.3410818585097 and emission 322.28729980862045
Firm S1_F2 has output 324.34108185850965 and emission 322.6477832848179
Firm S1_F3 has output 324.3410818585096 and emission 322.85053183406694
Firm S1_F4 has output 324.3410818585096 and emission 322.6070730395488
Firm S1_F5 has output 324.34108185850965 and emission 322.3423735563338
Firm S2_F1 has output 486.51162278776434 and emission 485.02045983338184
Firm S2_F2 has output 486.5116227877644 and emission 484.63120123386693
Firm S2_F3 has output 486.5116227877643 and emission 483.9983155420201
Firm S2_F4 has output 486.51162278776434 and emission 484.40374277942396
Firm S2_F5 has output 486.5116227877644 and emission 484.2168659336088
Firm S3_F1 has output 619.7673608062703 and emission 617.1392706216209
Firm S3_F2 has output 619.7673608062704 and emission 617.4168495506475
Firm S3_F3 has output 619.7673608062703 and emission 617.8466279304777
Firm S3_F4 has output 619.7673608062702 and e

(True, 3.56773977472358e-7)

In [5]:
coeff = RegulatorETS.exp7_find_coefficient_for_free_allocation(print_output=True, percentage=0.4)

Lower bound: 0, Upper bound: 1.0, Current bound: 0.5, Total free allocation: 5672.916597877253
Lower bound: 0.5, Upper bound: 1.0, Current bound: 0.75, Total free allocation: 2699.5655630242736
Lower bound: 0.75, Upper bound: 1.0, Current bound: 0.875, Total free allocation: 4146.239876985111
Lower bound: 0.875, Upper bound: 1.0, Current bound: 0.9375, Total free allocation: 4898.708876592799
Lower bound: 0.9375, Upper bound: 1.0, Current bound: 0.96875, Total free allocation: 5282.975558502469
Lower bound: 0.96875, Upper bound: 1.0, Current bound: 0.984375, Total free allocation: 5477.22102522784
Lower bound: 0.984375, Upper bound: 1.0, Current bound: 0.9921875, Total free allocation: 5574.88552714837
Lower bound: 0.9921875, Upper bound: 1.0, Current bound: 0.99609375, Total free allocation: 5623.854985467409


In [6]:
for sector in RegulatorETS.sector_registry.values():
    sector.free_emission_multiplier *= coeff

In [7]:
RegulatorETS.optimization_with_positive_constraints_ab(print_output= True, gurobi_print=False)

Optimal solution found
Firm S1_F1 has output 324.2090854190903 and emission 322.15667519002074
Firm S1_F2 has output 324.2090854190901 and emission 322.5168204315117
Firm S1_F3 has output 324.2090854190902 and emission 322.7194072900629
Firm S1_F4 has output 324.2090854190902 and emission 322.47608981369075
Firm S1_F5 has output 324.2090854190902 and emission 322.21177413862006
Firm S2_F1 has output 486.22589497561376 and emission 484.7355470352176
Firm S2_F2 has output 486.22589497561364 and emission 484.34649666795724
Firm S2_F3 has output 486.22589497561364 and emission 483.71405943372264
Firm S2_F4 has output 486.22589497561364 and emission 484.1193628998892
Firm S2_F5 has output 486.2258949756138 and emission 483.9325527424143
Firm S3_F1 has output 619.9224386083678 and emission 617.295763048605
Firm S3_F2 has output 619.9224386083679 and emission 617.5733158090355
Firm S3_F3 has output 619.9224386083679 and emission 618.0027092086351
Firm S3_F4 has output 619.9224386083678 and em

<gurobi.Model MIP instance positive_constraints_ab: 91 constrs, 61 vars, Parameter changes: Username=(user-defined), OutputFlag=0>

In [8]:
consumer_surplus = 0
for sector in RegulatorETS.sector_registry.values():
    consumer_surplus += sector.get_consumer_surplus()
print(consumer_surplus)
# 721857.091272771

763632.636748438


In [9]:
for sector in RegulatorETS.sector_registry.values():
    print(f"{sector.name} has a free emission multiplier {sector.free_emission_multiplier}")

Steel has a free emission multiplier 0.796875
Cement has a free emission multiplier 0.8466796875
Paper has a free emission multiplier 0.0498046875
Chemicals has a free emission multiplier 0.896484375
Automotive has a free emission multiplier 0.7470703125
Textiles has a free emission multiplier 0.0


In [10]:
for firm in RegulatorETS.firm_registry.values():
    firm.calculate_profit_components()

In [11]:
# =======================
# 1. Configuration Variables
# =======================
RUN_NAME = 'run_ULM'  # Unique identifier for the current run
BASE_PATH = 'C:/Users/Kostas/Documents/GitHub/Diplomatiki_kwpap_step_1/2024 Project v2/data/exp7'  # Fixed base path for data storage

# =======================
# 5. Data Saving
# =======================

def save_data(regulator, run_name, base_path):
    """
    Saves the data from the regulator object into CSV files.

    Parameters:
    - regulator (Regulator): The regulator instance containing all data.
    - run_name (str): Unique identifier for the current run.
    - base_path (str): Fixed base directory for data storage.
    """
    # Define run-specific paths
    run_path = os.path.join(base_path, run_name)
    data_dir = os.path.join(run_path, 'data')
    summaries_dir = os.path.join(run_path, 'summaries')

    # Create directories if they don't exist
    os.makedirs(data_dir, exist_ok=True)
    os.makedirs(summaries_dir, exist_ok=True)

    # 1. Extract Regulator Data
    regulator_data = {
        'id': [regulator.id],
        'name': [regulator.name],
        'permit_price': [regulator.permit_price],
        'emission_cap': [regulator.emission_cap],
        'BAU_emissions': [regulator.BAU_emissions],
        'BAU_profit': [regulator.BAU_profit]
    }

    df_regulator = pd.DataFrame(regulator_data)

    # Save Regulator DataFrame to CSV
    regulator_csv = os.path.join(data_dir, f'{run_name}_regulator.csv')
    df_regulator.to_csv(regulator_csv, index=False)
    print(f"Regulator data saved to {regulator_csv}.")

    # 2. Extract Countries Data
    countries = regulator.country_registry.values()
    country_list = []
    for country in countries:
        country_list.append({
            'id': country.id,
            'name': country.name,
            'size': country.size,
            'GDP_per_capita': country.GDP_per_capita,
            'population': country.population,
            'industry_percentage': country.industry_percentage,
            'GDP': country.GDP
        })

    df_countries = pd.DataFrame(country_list)

    # Save Countries DataFrame to CSV
    countries_csv = os.path.join(data_dir, f'{run_name}_countries.csv')
    df_countries.to_csv(countries_csv, index=False)
    print(f"Countries data saved to {countries_csv}.")

    # 3. Extract Sectors Data
    sectors = regulator.sector_registry.values()
    sector_list = []
    for sector in sectors:
        sector_list.append({
            'id': sector.id,
            'name': sector.name,
            'price_demand_function': str(sector.price_demand_function),
            'free_emission_multiplier': sector.free_emission_multiplier,
            'consumer_surplus': sector.consumer_surplus  # Include consumer_surplus
        })

    df_sectors = pd.DataFrame(sector_list)

    # Save Sectors DataFrame to CSV
    sectors_csv = os.path.join(data_dir, f'{run_name}_sectors.csv')
    df_sectors.to_csv(sectors_csv, index=False)
    print(f"Sectors data saved to {sectors_csv}.")

    # 4. Extract Firms Data
    firm_list = []
    for sector in sectors:
        for firm in sector.firms:
            firm_list.append({
                'id': firm.id,
                'name': firm.name,
                'sector_id': firm.sector.id,
                'country_id': firm.country.id,
                'production_cost_function': str(firm.production_cost_function),
                'abatement_cost_function': str(firm.abatement_cost_function),
                'actual_output': firm.actual_output,
                'emission': firm.emission,
                'profit': firm.profit,
                'BAU_output': firm.BAU_output,
                'BAU_emission': firm.BAU_emission,
                'BAU_profit': firm.BAU_profit,
                'exp7_free_allocation': firm.exp7_free_allocation,
                'abatement': firm.abatement,
                'permits_costs': firm.permits_costs,
                'sales': firm.sales,
                'permits_used': firm.permits_used,
                'free_permits': firm.free_permits,
                'permits_bought': firm.permits_bought
            })

    df_firms = pd.DataFrame(firm_list)

    # Save Firms DataFrame to CSV
    firms_csv = os.path.join(data_dir, f'{run_name}_firms.csv')
    df_firms.to_csv(firms_csv, index=False)
    print(f"Firms data saved to {firms_csv}.")

    # 5. Save Summary Statistics
    # Regulator Summary
    regulator_summary_csv = os.path.join(summaries_dir, f'{run_name}_summary_regulator.csv')
    df_regulator.describe().to_csv(regulator_summary_csv)
    print(f"Regulator summary saved to {regulator_summary_csv}.")

    # Sectors Summary
    sectors_summary_csv = os.path.join(summaries_dir, f'{run_name}_summary_sectors.csv')
    df_sectors.describe(include='all').to_csv(sectors_summary_csv)
    print(f"Sectors summary saved to {sectors_summary_csv}.")

    # Countries Summary
    countries_summary_csv = os.path.join(summaries_dir, f'{run_name}_summary_countries.csv')
    df_countries.describe().to_csv(countries_summary_csv)
    print(f"Countries summary saved to {countries_summary_csv}.")

    # Firms Summary
    firms_summary_csv = os.path.join(summaries_dir, f'{run_name}_summary_firms.csv')
    df_firms.describe(include='all').to_csv(firms_summary_csv)
    print(f"Firms summary saved to {firms_summary_csv}.")

    print("\nAll data and summaries have been saved successfully.")

# =======================
# 6. Execute Data Saving
# =======================
save_data(RegulatorETS, RUN_NAME, BASE_PATH)

Regulator data saved to C:/Users/Kostas/Documents/GitHub/Diplomatiki_kwpap_step_1/2024 Project v2/data/exp7\run_ULM\data\run_ULM_regulator.csv.
Countries data saved to C:/Users/Kostas/Documents/GitHub/Diplomatiki_kwpap_step_1/2024 Project v2/data/exp7\run_ULM\data\run_ULM_countries.csv.
Sectors data saved to C:/Users/Kostas/Documents/GitHub/Diplomatiki_kwpap_step_1/2024 Project v2/data/exp7\run_ULM\data\run_ULM_sectors.csv.
Firms data saved to C:/Users/Kostas/Documents/GitHub/Diplomatiki_kwpap_step_1/2024 Project v2/data/exp7\run_ULM\data\run_ULM_firms.csv.
Regulator summary saved to C:/Users/Kostas/Documents/GitHub/Diplomatiki_kwpap_step_1/2024 Project v2/data/exp7\run_ULM\summaries\run_ULM_summary_regulator.csv.
Sectors summary saved to C:/Users/Kostas/Documents/GitHub/Diplomatiki_kwpap_step_1/2024 Project v2/data/exp7\run_ULM\summaries\run_ULM_summary_sectors.csv.
Countries summary saved to C:/Users/Kostas/Documents/GitHub/Diplomatiki_kwpap_step_1/2024 Project v2/data/exp7\run_ULM\s

In [12]:
# import optuna

# # Define the objective function for optimization
# def objective(trial):
#     # Define possible multiplier values from 0 to 1 with 0.05 increments
#     multiplier_values = [i * 0.05 for i in range(21)]
    
#     # Assign a multiplier to each sector using sector names as keys
#     multipliers = {}
#     for sector in RegulatorETS.sector_registry.values():
#         multipliers[sector.name] = trial.suggest_categorical(
#             f"multiplier_{sector.name}", multiplier_values
#         )
    
#     try:
#         # Set the free_emission_multiplier for each sector
#         for sector in RegulatorETS.sector_registry.values():
#             sector.free_emission_multiplier = multipliers[sector.name]
        
#         # Find the scaling coefficient to maintain the emission cap percentage
#         coeff = RegulatorETS.exp7_find_coefficient_for_free_allocation(
#             print_output=False, percentage=0.4
#         )
        
#         # Check if coefficient was found
#         if coeff is None:
#             raise ValueError("Scaling coefficient not found.")
        
#         # Scale the free_emission_multiplier for each sector
#         for sector in RegulatorETS.sector_registry.values():
#             sector.free_emission_multiplier *= coeff
        
#         # Run the economic simulation with the updated multipliers
#         RegulatorETS.optimization_with_positive_constraints_ab(
#             print_output=False, gurobi_print=False
#         )
#         if (any (sector.free_emission_multiplier > 1 for sector in RegulatorETS.sector_registry.values())):
#             return float('inf')  # Penalize invalid configurations
        
#         # Calculate the total consumer surplus
#         consumer_surplus = sum(
#             sector.get_consumer_surplus() for sector in RegulatorETS.sector_registry.values()
#         )
        
#         # Since Optuna minimizes the objective, return the negative consumer surplus
#         return -consumer_surplus
    
#     except KeyError as e:
#         print(f"KeyError encountered: {e}")
#         return float('inf')  # Penalize invalid configurations
#     except Exception as e:
#         print(f"An error occurred: {e}")
#         return float('inf')  # Penalize invalid configurations

# # Create an Optuna study to maximize consumer surplus
# study = optuna.create_study(direction='minimize')

# # Optimize the objective function
# study.optimize(objective, n_trials=100, show_progress_bar=True)

# # Retrieve the best multipliers found
# best_multipliers = {
#     sector_name.replace("multiplier_", ""): value
#     for sector_name, value in study.best_params.items()
# }

# # Display the optimal free_emission_multipliers for each sector
# print("Optimal free_emission_multipliers:")
# for sector, multiplier in best_multipliers.items():
#     print(f"{sector}: {multiplier}")

# # Optionally, set the best multipliers and calculate the final consumer surplus
# for sector in RegulatorETS.sector_registry.values():
#     sector.free_emission_multiplier = best_multipliers[sector.name]
    
# # Apply the scaling coefficient
# try:
#     coeff = RegulatorETS.exp7_find_coefficient_for_free_allocation(
#         print_output=False, percentage=0.4
#     )
#     if coeff is None:
#         raise ValueError("Scaling coefficient not found.")
#     for sector in RegulatorETS.sector_registry.values():
#         sector.free_emission_multiplier *= coeff
    
#     # Run the optimization with the best multipliers
#     RegulatorETS.optimization_with_positive_constraints_ab(
#         print_output=True, gurobi_print=False
#     )
    
#     # Calculate and print the maximum consumer surplus
#     consumer_surplus = sum(
#         sector.get_consumer_surplus() for sector in RegulatorETS.sector_registry.values()
#     )
#     print(f"Maximum Consumer Surplus: {consumer_surplus}")
    
# except Exception as e:
#     print(f"An error occurred during the final simulation: {e}")

# old run 1,0,0,1,0.9,0